In [100]:
pip install fitz

^C


In [101]:
pip install pymupdf

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [102]:
import fitz

pdf_doc = fitz.open("Poemas_Antonio_Machado.docx")

poemas = []

for page in pdf_doc:
  text = page.get_text()
  poemas.extend(text.strip().split("\n\n"))

poemas

['Está en la sala familiar, sombría,\ny entre nosotros, el querido hermano\nque en el sueño infantil de un claro día\nvimos partir hacia un país lejano.\nHoy tiene ya las sienes plateadas,\nun gris mechón sobre la angosta frente;\ny la fría inquietud de sus miradas\nrevela un alma casi toda ausente.\nDeshójanse las copas otoñales\ndel parque mustio y viejo.\nLa tarde, tras los húmedos cristales,\nse pinta, y en el fondo del espejo.\nEl rostro del hermano se ilumina\nsuavemente. ¿Floridos desengaños\ndorados por la tarde que declina?\n¿Ansias de vida nueva en nuevos años?\n¿Lamentará la juventud perdida?\nLejos quedó —la pobre loba— muerta.\n¿La blanca juventud nunca vivida\nteme, que ha de cantar ante su puerta?\n¿Sonríe al sol de oro,\nde la tierra de un sueño no encontrada;',
 'y ve su nave hender el mar sonoro,\nde viento y luz la blanca vela henchida?\nÉl ha visto las hojas otoñales,\namarillas, rodar, las olorosas\nramas del eucalipto, los rosales\nque enseñan otra vez sus blancas

In [103]:
import re

def insert_spaces_before_after_newline(text):
    # Utiliza expresiones regulares para encontrar todas las ocurrencias de \n
    pattern = re.compile(r'(?<=\n)|(?=\n)')
    # Inserta un espacio antes y después de cada ocurrencia de \n
    modified_text = pattern.sub(' ', text)
    return modified_text

# Meter espacios alrededor de los saltos de línea
for i, poema in enumerate(poemas):
  poemas[i] = insert_spaces_before_after_newline(poemas[i])

poemas

['Está en la sala familiar, sombría, \n y entre nosotros, el querido hermano \n que en el sueño infantil de un claro día \n vimos partir hacia un país lejano. \n Hoy tiene ya las sienes plateadas, \n un gris mechón sobre la angosta frente; \n y la fría inquietud de sus miradas \n revela un alma casi toda ausente. \n Deshójanse las copas otoñales \n del parque mustio y viejo. \n La tarde, tras los húmedos cristales, \n se pinta, y en el fondo del espejo. \n El rostro del hermano se ilumina \n suavemente. ¿Floridos desengaños \n dorados por la tarde que declina? \n ¿Ansias de vida nueva en nuevos años? \n ¿Lamentará la juventud perdida? \n Lejos quedó —la pobre loba— muerta. \n ¿La blanca juventud nunca vivida \n teme, que ha de cantar ante su puerta? \n ¿Sonríe al sol de oro, \n de la tierra de un sueño no encontrada;',
 'y ve su nave hender el mar sonoro, \n de viento y luz la blanca vela henchida? \n Él ha visto las hojas otoñales, \n amarillas, rodar, las olorosas \n ramas del eucali

In [104]:
# Calcular el total de palabras en todos los poemas
total_palabras = sum(len(poema.split()) for poema in poemas)

# Calcular la media de palabras en total
media_palabras_total = total_palabras / len(poemas)

print("Media de palabras en total:", media_palabras_total)

Media de palabras en total: 109.71428571428571


In [105]:
from tensorflow import keras
from tensorflow.keras import layers
import string
import numpy as np
import tensorflow as tf

max_length = round(media_palabras_total)
max_tokens = 20000  # Tamaño del vocabulario
batch_size = 4      # Tamaño del lote
embed_dim = 256     # Dimensión del embedding
num_heads = 2       # Número de cabezas del MultiHead
dense_dim = 32      # Nº de neuronas de la capa densa
EPOCHS = 30         # Nº de épocas

In [106]:
def split_text(text):
    # Calcula la longitud total del texto
    text_length = len(text)

    # Encuentra el índice medio del texto
    middle_index = text_length // 2

    # Retrocede desde el índice medio hasta el inicio del texto y encuentra el espacio más cercano
    start_index = text.rfind(' ', 0, middle_index)

    # Retrocede desde el índice medio hasta el final del texto y encuentra el espacio más cercano
    end_index = text.find(' ', middle_index)

    # Si no se encontraron espacios antes o después del índice medio, ajusta los índices para dividir entre palabras
    if start_index == -1:
        start_index = 0
    if end_index == -1:
        end_index = text_length

    # Divide el texto en dos partes
    first_half = text[:end_index]
    second_half = text[end_index:]

    return first_half, second_half

In [107]:
poema_pairs = []
for poema in poemas:
  poema_mitad1, poema_mitad2 = split_text(poema)
  poema_pairs.append((poema_mitad1, "[start]" + poema_mitad2 + " [end]"))

In [108]:
import random
random.shuffle(poema_pairs)
# 15% validación
num_val_samples = int(0.15 * len(poema_pairs))
# 70% entrenamiento
num_train_samples = len(poema_pairs) - 2 * num_val_samples
train_pairs = poema_pairs[:num_train_samples]
val_pairs = poema_pairs[num_train_samples:num_train_samples + num_val_samples]
# 15% test o pruebas
test_pairs = poema_pairs[num_train_samples + num_val_samples:]

In [109]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [110]:
# Prepara una función de estandarización de cadenas
# personalizada para la capa TextVectorization en
# español: conserva [ y ] pero elimina ¿ y ¡ (así como
# todos los demás caracteres de cadenas.puntuación)
# Poemas
strip_chars = string.punctuation + "¿" + "¡"
strip_chars = strip_chars.replace("\\", "")

# Poemas con [start] y [end]
strip_chars2 = string.punctuation + "¿" + "¡"
strip_chars2 = strip_chars2.replace("[", "")
strip_chars2 = strip_chars2.replace("]", "")
strip_chars2 = strip_chars2.replace("\\", "")

# Poemas
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length,
    standardize=custom_standardization,
)

# Poemas con [start] y [end]
def custom_standardization_2(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars2)}]", "")

text_vectorization_start_end = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    # Generamos poemas que tengan un token
    # adicional, ya que necesitaremos compensar la oración
    # en un paso durante el entrenamiento
    output_sequence_length=max_length + 1,
    standardize=custom_standardization_2,
)

train_poemas = [pair[0] for pair in train_pairs]
train_poemas_start_end = [pair[1] for pair in train_pairs]

# Aprende el vocabulario de cada poema
text_vectorization.adapt(train_poemas)
text_vectorization_start_end.adapt(train_poemas_start_end)

In [111]:
def format_dataset(p, p_s_e):
    p = text_vectorization(p)
    p_s_e = text_vectorization_start_end(p_s_e)
    return ({
        "poema": p,
        # El poema de entrada
        # no incluye el último token para
        # mantener las entradas y los
        # objetivos en la misma longitud.
        "poema_start_end": p_s_e[:, :-1],
    # El poema objetivo está un
    # paso por delante. Ambos siguen siendo
    # de la misma longitud
    }, p_s_e[:, 1:])

def make_dataset(pairs):
    poema_texts, poema_start_end_texts = zip(*pairs)
    poema_texts = list(poema_texts)
    poema_start_end_texts = list(poema_start_end_texts)
    dataset = tf.data.Dataset.from_tensor_slices((poema_texts, poema_start_end_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    # Utilizamos el almacenamiento en caché en memoria
    # para acelerar el preprocesamiento
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [112]:
for inputs, targets in train_ds.take(1):
  print(f"inputs['poema'].shape: {inputs['poema'].shape}")
  print(f"inputs['poema_start_end'].shape: {inputs['poema_start_end'].shape}")
  print(f"targets.shape: {targets.shape}")

inputs['poema'].shape: (3, 110)
inputs['poema_start_end'].shape: (3, 110)
targets.shape: (3, 110)


In [113]:
import tensorflow as tf

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding
        # será 2D, pero la capa de atención espera
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [114]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        # Este atributo asegura que la capa propagará
        # su máscara de entrada a sus salidas; el
        # enmascaramiento en Keras es explícitamente
        # opt-in. Si pasa una máscara a una capa que
        # no implementa compute_mask() y que no expone
        # este atributo support_masking, es un error.
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [115]:
class PositionalEmbedding(layers.Layer):
    # Una desventaja de las incrustaciones de posición es que
    # la longitud de la secuencia debe conocerse de antemano
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Prepara una capa de embedding para los índices de token.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim, mask_zero = True)
        self.position_embeddings = layers.Embedding(
            # Y otro para las posiciones te tokens
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # Agrega ambos vectores embeddings juntos
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Al igual que la capa de embedding,
        # esta capa debería poder generar una
        # máscara para que podamos ignorar los
        # ceros de relleno en las entradas.
        # El framework llamará automáticamente
        # al método compute_mask y la máscara
        # se propagará a la siguiente capa.
        return tf.math.not_equal(inputs, 0)

    # Implementamos la serialización para que
    # podamos guardar el modelo.
    def get_config(self):
        config = super().get_config()
        # config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [116]:
def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        # Genere una matriz de forma (sequence_length, sequence_length)
        # con 1 en una mitad y 0 en la otra
        mask = tf.cast(i >= j, dtype="int32")
        # Lo replicamos a lo largo del eje del lote para obtener una matriz
        # de forma  (batch_size, sequence_length, sequence_length)
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

In [117]:
def call(self, inputs, encoder_outputs, mask=None):
        # Recupera la máscara causal
        causal_mask = self.get_causal_attention_mask(inputs)
        # Prepara la máscara de entrada (que describe las
        # ubicaciones de relleno en la secuencia de destino)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            # Fusiona las dos máscaras juntas
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            # Pasamos la máscara causal a la primera capa de atención,
            # que realiza la self-attention sobre la secuencia de destino.
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            # Pasamos la máscara combinada a la segunda
            # capa de atención, que relaciona la secuencia
            # de origen con la secuencia de destino
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [118]:
# Transformer encoder
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name = "poema")
x = PositionalEmbedding(max_length, max_tokens, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

# Transformer decoder
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name = "poema_start_end")
x = PositionalEmbedding(max_length, max_tokens, embed_dim)(decoder_inputs)
# Codificamos la oración objetivo y la combinamos con la oración fuente codificada
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)

# Predecimos una palabra para cada posición de salida
decoder_outputs = layers.Dense(max_tokens, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

transformer.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
transformer.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 poema (InputLayer)          [(None, None)]               0         []                            
                                                                                                  
 poema_start_end (InputLaye  [(None, None)]               0         []                            
 r)                                                                                               
                                                                                                  
 positional_embedding_10 (P  (None, None, 256)            5148160   ['poema[0][0]']               
 ositionalEmbedding)                                                                              
                                                                                            

In [119]:
transformer.fit(train_ds, epochs=EPOCHS, validation_data=val_ds)

Epoch 1/30
9/9 [==============================] - 19s 2s/step - loss: 9.3474 - accuracy: 0.0405 - val_loss: 8.6608 - val_accuracy: 0.0400
Epoch 2/30
9/9 [==============================] - 11s 1s/step - loss: 7.8927 - accuracy: 0.0552 - val_loss: 7.7293 - val_accuracy: 0.0400
Epoch 3/30
9/9 [==============================] - 6s 660ms/step - loss: 7.0401 - accuracy: 0.0531 - val_loss: 7.4971 - val_accuracy: 0.0400
Epoch 4/30
9/9 [==============================] - 6s 652ms/step - loss: 6.6736 - accuracy: 0.0698 - val_loss: 7.4188 - val_accuracy: 0.0800
Epoch 5/30
9/9 [==============================] - 7s 791ms/step - loss: 6.4250 - accuracy: 0.0815 - val_loss: 7.3824 - val_accuracy: 0.0771
Epoch 6/30
9/9 [==============================] - 6s 718ms/step - loss: 6.1707 - accuracy: 0.0992 - val_loss: 7.3292 - val_accuracy: 0.0800
Epoch 7/30
9/9 [==============================] - 6s 645ms/step - loss: 5.9546 - accuracy: 0.1149 - val_loss: 7.2728 - val_accuracy: 0.0857
Epoch 8/30
9/9 [========

In [120]:
import numpy as np
poema_vocab = text_vectorization_start_end.get_vocabulary()
spa_index_lookup = dict(zip(range(len(poema_vocab)), poema_vocab))
max_decoded_sentence_length = max_length

def decode_sequence(input_sentence):
    tokenized_input_sentence = text_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = text_vectorization_start_end(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        # Muestra el siguiente token
        sampled_token_index = np.argmax(predictions[0, i, :])
        # Convertimos la siguiente predicción del token en una
        # cadena y la agregamos a la oración generada
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        # Condición de salida
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(5):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
la mirra y el incienso salmodiarán su olor; 
 exhalarán su fresco perfume los rosales, 
 bajo la paz en sombra del tibio huerto en flor. 
 Al grave acorde lento de música y aroma, 
 la sola y vieja y noble razón de mi rezar 
 levantará su vuelo suave de paloma, 
 y la palabra blanca se elevará al altar. 
 Daba el reloj las doce... y eran doce 
 golpes de azada
[start] abril el mar hierve y ríe con olas azules y espumas de la tristeza del sol yerto y humilde en sombra el agua sueña donde el mar lactescente el mar rutilante que ríe en sus liras de plata sus risas azules hierve y ríe el mar el aire parece que duerme encantado en el blanco sendero los montes [end]
-
Lejos de tu jardín quema la tarde 
 inciensos de oro en purpurinas llamas, 
 tras el bosque de cobre y de ceniza. 
 En tu jardín hay dalias. 
 ¡Malhaya tu jardín!... Hoy me parece 
 la obra de un peluquero, 
 con esa pobre palmerilla enana, 
 y ese cuadro de mirtos recortados... 
 y el naranjito en su tonel... El agua 
 de la